In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib



In [3]:
# Load the dataset
df = pd.read_csv("Error Annotated Corpus.csv")

# Handle missing values: Drop rows with NaN values
df = df.dropna(subset=["Error word & consecutive word", "Corrected words & its"])

# Check for missing values after cleaning
print("Missing values after cleaning:")
print(df.isna().sum())

Missing values after cleaning:
Error word & consecutive word    0
Corrected words & its            0
Annotation                       0
dtype: int64


In [8]:
df.head(15)

,Error word & consecutive word,Corrected words & its,Annotation
0,10கனநீர் உலைகள்,10கணநீர் உலைகள்,வேற்றெழுத்து
1,அகளவிரித்து,அகலவிரித்து,வேற்றெழுத்து
2,அணைத்து ஊழியர்களுக்குமான,அனைத்து ஊழியர்களுக்குமான,வேற்றெழுத்து
3,அதள பாதாளதுக்கு,அதல பாதாளத்துக்கு,வேற்றெழுத்து
4,அதற்குறிய தீர்வுகளை,அதற்குரிய தீர்வுகளை,வேற்றெழுத்து
5,அதற்கேற்றார்போல அதன்,அதற்கேற்றாற்போல அதன்,வேற்றெழுத்து
6,அந்நியமாக நடந்தது,"அன்னியமாக,அன்னியமாக்க நடந்தது",வேற்றெழுத்து
7,அம்பிலாந்துறை,அம்பிளாந்துறை,வேற்றெழுத்து
8,அரவனைத்துச் செல்கின்ற,அரவணைத்துச் செல்கின்ற,வேற்றெழுத்து
9,அருவறுப்பாக நினைக்கிறார்கள்,அறுவறுப்பாக நினைக்கிறார்கள்,வேற்றெழுத்து


In [4]:
# Prepare the data: Input features (incorrect sentences) and output labels (corrected sentences)
X = df["Error word & consecutive word"]  # Input features
y = df["Corrected words & its"]  # Output labels

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\S+', ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)

# Train a Random Forest Classifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_vec, y_train)



RandomForestClassifier(random_state=42)

In [5]:
# Make predictions on the validation set
y_val_pred = model.predict(X_val_vec)

# Evaluate the model performance
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Make predictions on the test set
y_test_pred = model.predict(X_test_vec)

# Evaluate the model on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the trained model and vectorizer for future use
joblib.dump(model, 'spell_check_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')



Validation Accuracy: 0.62%
Test Accuracy: 0.90%


['vectorizer.pkl']

In [ ]:
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained model and vectorizer
model = joblib.load('spell_check_model.pkl')
vectorizer = joblib.load('tamil_vectorizer.pkl')

def correct_tamil_paragraph(paragraph, model, vectorizer):
    """
    Corrects errors in a Tamil paragraph using the trained model.
    """
    # Tokenize the paragraph, preserving punctuation and spaces
    tokens = re.findall(r'\S+|[.,!?]', paragraph)

    corrected_tokens = []
    for token in tokens:
        # Check if token is a Tamil word (ignoring punctuation)
        if re.match(r'^[\u0B80-\u0BFF]+$', token):  # Tamil Unicode range
            # Transform the token using the vectorizer
            token_vec = vectorizer.transform([token])

            # Predict the correction if the token is in the vectorizer's vocabulary
            try:
                corrected_token = model.predict(token_vec)[0]
                corrected_tokens.append(corrected_token)
            except:
                corrected_tokens.append(token)  # Keep the original token if it cannot be corrected
        else:
            corrected_tokens.append(token)  # Add punctuation or non-Tamil words as is

    # Reconstruct the corrected paragraph
    corrected_paragraph = ''.join(
        [token if token in ".,!?" else f" {token}" for token in corrected_tokens]
    ).strip()

    return corrected_paragraph

# Example input paragraph in Tamil
input_paragraph = "இந்த காலை அவன் மிகவும் சந்தோஷமாக இருந்தான் அவன் செல்ல தயாராக இருந்தான் அவன் பெற்றோருடன் உணவு பறிமாறிக் கொண்டிருந்தான் மற்றும் செல்லும் போது அவன் புதிய நண்பர்கள் சந்திக்க ஆவலுடன் இருந்தார்கள் படிப்பில் மிகவும் கவனம் செலுத்துவார் மற்றும் நாளும் அதிகம் பெற்றுக்கொள்ள விரும்புவோர்"

# Correct the Tamil paragraph
corrected_paragraph = correct_tamil_paragraph(input_paragraph, model, vectorizer)

print("\nCorrected Paragraph:")
print(corrected_paragraph)


In [1]:
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained model and vectorizer
model = joblib.load('spell_check_model.pkl')
vectorizer = joblib.load('tamil_vectorizer.pkl')

def correct_tamil_paragraph(paragraph, model, vectorizer):
    """
    Corrects errors in a Tamil paragraph using the trained model.
    """
    # Tokenize the paragraph, preserving punctuation and spaces
    tokens = re.findall(r'\S+|[.,!?]', paragraph)

    corrected_tokens = []
    for token in tokens:
        # Check if token is a Tamil word (ignoring punctuation)
        if re.match(r'^[\u0B80-\u0BFF]+$', token):  # Tamil Unicode range
            # Transform the token using the vectorizer
            token_vec = vectorizer.transform([token])

            # Predict the correction if the token is in the vectorizer's vocabulary
            try:
                corrected_token = model.predict(token_vec)[0]
                corrected_tokens.append(corrected_token)
            except:
                corrected_tokens.append(token)  # Keep the original token if it cannot be corrected
        else:
            corrected_tokens.append(token)  # Add punctuation or non-Tamil words as is

    # Reconstruct the corrected paragraph
    corrected_paragraph = ''.join(
        [token if token in ".,!?" else f" {token}" for token in corrected_tokens]
    ).strip()

    return corrected_paragraph

# Example input paragraph in Tamil
input_paragraph =  "இந்த காலை அவன் மிகவும் சந்தஷமாக இருந்தான். அவன் செலல தயராக இருந்தான். அவன் பெற்றோரகளுடன் உணவு பரிமாறிக கொண்டிருந்தான். மற்றும் செலலும் போது, புதிய நண்பர்களைச் சந்தக்க ஆவலுடன் இருந்தாகளை. படிப்பில் மிகவும் கவனம் செலத்துவா, மற்றும் நாளும் அதிகம் கற்றுக்கொளள வரும்புவா."

# Correct the Tamil paragraph
corrected_paragraph = correct_tamil_paragraph(input_paragraph, model, vectorizer)



Corrected Paragraph:
இந்த காலை அவன் மிகவும் சந்தோஷமாக இருந்தான். அவன் செல்ல தயாராக இருந்தான். அவன் பெற்றோருடன் உணவு பறிமாறிக் கொண்டிருந்தான். மற்றும் செல்லும் போது புதிய நண்பர்கள் சந்திக்க ஆவலுடன் இருந்தார்கள். படிப்பில் மிகவும் கவனம் செலுத்துவார், மற்றும் நாளும் அதிகம் பெற்றுக்கொள்ள விரும்புவோர்
